In [ ]:
from datasets import load_dataset
from qdrant_client import models,QdrantClient

ds = load_dataset("arbml/Hadith")


In [ ]:
ds['train'][0]

In [ ]:
documents = ds['train']
documents[0]

In [ ]:
client = QdrantClient(":memory:")

In [ ]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("jinaai/jina-colbert-v2")
docs = [
    "ColBERT is a novel ranking model that adapts deep LMs for efficient retrieval.",
    "Jina-ColBERT is a ColBERT-style model but based on JinaBERT so it can support both 8k context length, fast and accurate retrieval.",
]
RAG.index(docs, index_name="demo")
query = "What does ColBERT do?"
results = RAG.search(query)

In [ ]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("silma-ai/silma-embeddding-sts-v0.1", device="cuda")

# sentences = [
#     "ذكر متوازن بعناية يقف على قدم واحدة بالقرب من منطقة شاطئ المحيط النظيفة",
#     "رجل يقدم عرضاً",
#     "هناك رجل بالخارج قرب الشاطئ",
#     "رجل يجلس على أريكه",
# ]
# embeddings = model.encode(sentences)

# similarities = model.similarity(embeddings, embeddings)
# print(similarities.shape)
# # [4, 4]

In [ ]:
client.create_collection(
    collection_name="hadith_slima",
    vectors_config=models.VectorParams(
        size=model.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,
        )
    )


In [ ]:
client.upload_points(
    collection_name="hadith_slima",
    points=[
        models.PointStruct(
            id=idx, vector=model.encode(doc["Text"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

In [ ]:
h_1_t = "من ترك صلاة مكتوبة متعمدا برئت منه ذمة الله"  # 40165
h_2_t = "من سن في الإسلام سنة حسنة"  # 37492
h_3_t = "الراحمون يرحمهم الرحمن"  # 25311
h_4_t = "مثل المؤنين في توادهم وتراحمهم كمثل الجسد الواحد"  # 36753
h_5_t = "ألا إن في الجسد مضغة إذا صلحت صلح الجسد كله"  # 81731
h_6_t = "إن العبد ليتكلم بالكلمة من سخط الله"  # 27672
h_7_t = "من تشبه بقوم فهو منهم"  # 23973
h_8_t = "اجتنبوا أم الخبائث"  # 5573
h_9_t = "ليس المؤمن باللعان و لا الطعان ولا الفاحش"  # 22751
h_10_t = "فضل العالم علي العابد كفضل القمر"  # 39828

In [ ]:
h_1 = "العهد الذي  بيننا وبينهم الصلاة فمن تركها فقد كفر"  # 78826
h_2 = "من دعا الي الهدي كان له من الأجور مثل أجور متبعه"  # 27900
h_3 = "من لا يرحم الناس لا يرحمه الله"  # 30040
h_4 = "المؤمن للمؤمن كالبنيان يشد بعضه بعضا"  # 37904
h_5 = "إن الله لا ينظر الي صوركم و لكن ينظرالي قلوبكم"  # 29642
h_6 = "من كان يؤمن بالله واليوم الأخر فليقل خيرا أو ليصمت"  # 25437
h_7 = "يحشر المرء مع من يحب"  #
h_8 = "لعن الله الخمر وشاربه وساقيها"  # 24563
h_9 = "سباب المسلم فسوق وقتاله كفر"  # 4037
h_10 = "من سلك طريقا يلتمس فيه علما"  # 27070

- omarmodel MiniALL
    1   -> good but exact
    2   -> not good
    3   -> exact but not in the first 3 results
    4   -> get it in the 9 result :(
    5   -> good 1 first
    6   -> not good
    7   -> not good
    8   -> not good
    9   -> around 5 similar results but not the first one
    10  -> bad 

- omar Arabic-Triplet-Matryoshka-V2
    1   -> very good
    2   -> very good
    3   -> very good
    4   -> very good
    5   -> very good
    6   -> very good
    7   -> very good
    8   -> very good
    9   -> very good
    10  -> very good
- Slima 
7/10 
not good at short sentences, not stable compared to Arabic-Triplet`
- Colbert-Jina 

- Arabic-Colbertv2

In [ ]:
hits = client.query_points(
    collection_name="hadith_slima",
    query=model.encode(h_10).tolist(),
    limit=15,
).points


for hit in hits:
    # print(hit.payload,"score:",hit.score,"hit_id",hit.id)
    print(hit.payload["Text"])
    print("score:", hit.score, "hit_id", hit.id)
    # print(hit)

In [ ]:
test_arbml = [(41032),(29640),(92508)]

In [ ]:
documents[92508]